In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
import nltk
nltk.download('words')
en_words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to /Users/nesara/nltk_data...
[nltk_data]   Package words is already up-to-date!


#### Extend stopwords

In [3]:
"""
(0,
  '0.016*"walk" + 0.012*"live" + 0.012*"area" + 0.010*"large" + 0.010*"great" '
  '+ 0.009*"flat" + 0.009*"floor" + 0.008*"away" + 0.008*"double" + '
  '0.008*"close"'),
 (1,
  '0.022*"private" + 0.017*"place" + 0.015*"access" + 0.015*"parking" + '
  '0.012*"space" + 0.012*"walk" + 0.010*"downtown" + 0.010*"location" + '
  '0.010*"close" + 0.009*"full"'),
 (2,
  '0.009*"check" + 0.008*"area" + 0.008*"large" + 0.007*"modern" + '
  '0.007*"rental" + 0.007*"build" + 0.007*"table" + 0.006*"dining" + '
  '0.006*"property" + 0.006*"living"'
"""

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(stopwords.words('french'))
stop_words.extend(stopwords.words('german'))
#stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'room', 'apartment', 'bedroom', 'home', 'bed', 'bathroom', 'kitchen',
#                  'NEIGHBORHOOD', 'neighborhood', 'apartment', "pie", "acce", "tre", "appartement", "commerce", "immeuble",
#                 ])

#### Load dataset

In [4]:
columns = ['ID', 'Name', 'City', 'Summary', 'Space', 'Description', 'Neighborhood Overview', 'House Rules']
df = pd.read_csv("../../data/airbnb-data-science/airbnb-listings.csv", usecols=columns, sep=';')
df = df[columns]
#df = pd.read_csv("../../data/airbnb-data-science/airbnb-listings-random100nyc.csv")
#df.head(2)

/Users/nesara/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def get_vibe_text(x):
    #print(x)
    text = ''
    #if not pd.isnull(x["Space"]):
    #    text += x["Space"]
    #if not pd.isnull(x["Description"]):
    #    text += x["Description"]
    if not pd.isnull(x["Neighborhood Overview"]):
        text += x["Neighborhood Overview"]
    #if not pd.isnull(x["House Rules"]):
    #    text += x["House Rules"]
    text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in en_words or not w.isalpha())
    return text
#df["vibe_text"] = df.apply(get_vibe_text, axis=1)
#df

#### Extract listings description

In [8]:
# Convert to list
#data = df.sample(n=10000, random_state=1)["vibe_text"].values.tolist()
df_nyc = df[df["City"]=="Amsterdam"].copy()
df_nyc.reset_index(inplace=True)
del df_nyc["index"]
df_nyc.reset_index(inplace=True)
df_nyc.rename(columns={"index":"Document_No"}, inplace=True)
df_nyc.head(2)

,Document_No,ID,Name,City,Summary,Space,Description,Neighborhood Overview,House Rules
0,0,17938550,light and cozy appartment,Amsterdam,"light and cozy appartment , 50mt2, 1 bedroom, ...",NaN,"light and cozy appartment , 50mt2, 1 bedroom, ...",NaN,- NO PARTIES -RESPECT THE NEIGHBOURS - NO NO...
1,1,5656694,Family house large rooftop terrace,Amsterdam,Our 160 sqm townhouse with a 61 sqm sunny roof...,This light and spacious 1900 townhouse with ro...,Our 160 sqm townhouse with a 61 sqm sunny roof...,NaN,We strongly prefer families with children. Smo...


In [9]:
data = df_nyc["Description"].values.tolist()
len(data)

14869

#### Tokenise words

In [10]:
def sent_to_words(sentences):
    i = 0
    for sentence in sentences:
        #if i%10000 == 0:
        #    print(i)
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        i+=1

data_words = list(sent_to_words(data))

#### Create bigram and trigram models

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#### Create the Dictionary and Corpus needed for Topic Modeling

In [13]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

#### Building the Topic Model

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

#### View the topics in LDA model 

In [159]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

[(0,
  '0.034*"walk" + 0.029*"restaurant" + 0.025*"block" + 0.024*"place" + '
  '0.023*"train" + 0.023*"away" + 0.022*"subway" + 0.021*"apartment" + '
  '0.021*"minute" + 0.019*"close"'),
 (1,
  '0.022*"floor" + 0.022*"apartment" + 0.020*"high" + 0.015*"view" + '
  '0.014*"new" + 0.014*"beautiful" + 0.014*"light" + 0.014*"bedroom" + '
  '0.014*"renovate" + 0.014*"building"'),
 (2,
  '0.047*"room" + 0.034*"bedroom" + 0.034*"apartment" + 0.023*"bed" + '
  '0.019*"kitchen" + 0.019*"bathroom" + 0.019*"full" + 0.019*"living" + '
  '0.014*"private" + 0.014*"guest"')]


#### Compute Model Perplexity and Coherence Score

In [234]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.526876964693229

Coherence Score:  0.5321964009159951


#### Visualize the topics-keywords

In [235]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/nesara/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.108124  0.229633       1        1  44.408150
0     -0.281935 -0.033097       2        1  36.562340
1      0.173811 -0.196535       3        1  19.029503, topic_info=     Category          Freq          Term         Total  loglift  logprob
term                                                                     
61    Default  21012.000000          room  21012.000000  30.0000  30.0000
20    Default  12461.000000          walk  12461.000000  29.0000  29.0000
60    Default  10638.000000    restaurant  10638.000000  28.0000  28.0000
138   Default   9342.000000         block   9342.000000  27.0000  27.0000
58    Default   8959.000000         place   8959.000000  26.0000  26.0000
72    Default   8568.000000         train   8568.000000  25.0000  25.0000
2     Default  10560.000000           bed  10560.000000  24.0000  24.0000
136   Default   8367.000000          away   8367.000000  23.0000  23.0000
299   Default   8282.000000        subway   8282.000000  22.0000  22.0000
57    Default   7636.000000        minute   7636.000000  21.0000  21.0000
81    Default  18331.000000       bedroom  18331.000000  20.0000  20.0000
30    Default   7235.000000         close   7235.000000  19.0000  19.0000
105   Default   8665.000000        living   8665.000000  18.0000  18.0000
10    Default   7264.000000         floor   7264.000000  17.0000  17.0000
48    Default   3921.000000          high   3921.000000  16.0000  16.0000
300   Default   6259.000000           bar   6259.000000  15.0000  15.0000
44    Default   6733.000000          good   6733.000000  14.0000  14.0000
109   Default   7224.000000  neighborhood   7224.000000  13.0000  13.0000
24    Default   9052.000000      bathroom   9052.000000  12.0000  12.0000
46    Default   6279.000000         guest   6279.000000  11.0000  11.0000
102   Default   9262.000000       kitchen   9262.000000  10.0000  10.0000
11    Default   9287.000000          full   9287.000000   9.0000   9.0000
129   Default   2898.000000          view   2898.000000   8.0000   8.0000
310   Default   4687.000000          line   4687.000000   7.0000   7.0000
18    Default   5555.000000          size   5555.000000   6.0000   6.0000
208   Default   2743.000000      renovate   2743.000000   5.0000   5.0000
312   Default   4377.000000          park   4377.000000   4.0000   4.0000
361   Default   3341.000000           new   3341.000000   3.0000   3.0000
236   Default   5326.000000      location   5326.000000   2.0000   2.0000
235   Default   3651.000000         light   3651.000000   1.0000   1.0000
...       ...           ...           ...           ...      ...      ...
512    Topic3    524.884277         patio    525.713867   1.6576  -5.9099
848    Topic3    481.184601      decorate    481.954315   1.6576  -5.9969
687    Topic3    423.444824        marble    424.163330   1.6575  -6.1247
1165   Topic3    422.018494     fireplace    422.767792   1.6574  -6.1281
849    Topic3    410.700256      designer    411.437592   1.6574  -6.1553
9      Topic3   1890.971313       feature   1896.449097   1.6563  -4.6283
411    Topic3    542.199829      boutique    543.481262   1.6568  -5.8775
208    Topic3   2692.112793      renovate   2743.823486   1.6402  -4.2750
22     Topic3   1507.479370     appliance   1593.263062   1.6038  -4.8549
693    Topic3   1429.132690           top   1549.667236   1.5782  -4.9083
361    Topic3   2786.751709           new   3341.961182   1.4775  -4.2405
235    Topic3   2743.964111         light   3651.161133   1.3735  -4.2560
207    Topic3    583.686035      recently    595.209229   1.6396  -5.8038
133    Topic3   2316.975098        window   3280.510254   1.3114  -4.4251
10     Topic3   4280.104492         floor   7264.119141   1.1302  -3.8114
25     Topic3   2759.331543     beautiful   4681.275391   1.1306  -4.2504
91     Topic3   1023.784180          face   1248.466675   1.4

#### Find dominant topic in each document

In [237]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        if i%1000==0:
            print(i)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

TypeError: '<' not supported between instances of 'int' and 'tuple'

In [215]:
len(df_dominant_topic)

19528

In [210]:
dff = df.reset_index()
dff.rename(columns={"index": "Document_No"}, inplace=True)
dff = dff[["Document_No", "ID"]]
dff = dff.set_index("Document_No").join(df_dominant_topic[["Document_No", "Dominant_Topic"]].set_index("Document_No")).reset_index()
dff.dropna(inplace=True)
dff.to_csv("df_topic.csv", index=False)
dff

,Document_No,ID,Dominant_Topic
0,0,17938550,2.0
1,1,5656694,2.0
2,2,15934725,2.0
3,3,13544842,2.0
4,4,14225052,2.0
5,5,10849177,2.0
6,6,13041979,2.0
7,7,11041048,0.0
8,8,13678942,2.0
9,9,13238423,2.0


In [212]:
dff[dff["ID"]==12363329]

,Document_No,ID,Dominant_Topic


In [214]:
df[df["ID"]==12363329]

,ID,Name,City,Summary,Space,Description,Neighborhood Overview,House Rules,vibe_text
130965,12363329,Near Columbia Univ&The City College,New York,You can use 14 feet x 13 feet clean and quiet ...,I offer you one of my rooms as a private bedro...,You can use 14 feet x 13 feet clean and quiet ...,Close to Columbia University (both main campus...,Please treat the space with respect and be con...,Close to University ( both main campus and med...


#### Topic distribution across documents

In [192]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",443.0,0.443
1,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",94.0,0.094
2,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",463.0,0.463
3,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",NaN,NaN
4,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",NaN,NaN
5,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",NaN,NaN
6,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",NaN,NaN
7,0.0,"walk, restaurant, block, place, train, away, s...",NaN,NaN
8,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",NaN,NaN
9,2.0,"room, bedroom, apartment, bed, kitchen, bathro...",NaN,NaN


## A/B Testing